In [1]:
import yfinance as yf
import openai
import json

In [2]:
client = openai.Client()

In [3]:
def retorna_cotacao_historica(ticker, periodo='1mo'):
    ticker_obj = yf.Ticker(f'{ticker}.SA')
    hist = ticker_obj.history(period=periodo)['Close']
    hist.index = hist.index.strftime('%Y-%m-%d')
    hist = round(hist, 2)
    
    if len(hist) > 30:
        slice_size = int(len(hist) / 30)
        hist = hist.iloc[::-slice_size][::-1]
        
    return hist.to_json()
    

In [5]:
retorna_cotacao_historica('ABEV3', '1d')

'{"2024-07-18":11.73}'

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "retorna_cotacao_historica",
            "description": "Retorna a cotação diária histórica para uma ação da bovespa",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "O ticker da ação. Exemplo: 'ABEV3' para ambev, 'PETR4' para petrobras, etc",
                    },
                    "periodo": {
                        "type": "string",
                        "description": "O período que será retornado de dados históricos \
									sendo '1mo' equivalente a um mês de dados, '1d' a \
             						1 dia e '1y' a 1 ano",
                        "enum": [
                            "1d",
                            "5d",
                            "1mo",
                            "6mo",
                            "1y",
                            "5y",
                            "10y",
                            "ytd",
                            "max",
                        ],
                    },
                },
            },
        },
    }
]

In [7]:
funcoes_disponiveis = {"retorna_cotacao_historica": retorna_cotacao_historica}

In [9]:
mensagens = [{"role": "user", "content": "Qaul é a cotação da ambev agora"}]

res = client.chat.completions.create(
    messages=mensagens,
    model="gpt-3.5-turbo",
    tools=tools,
    tool_choice='auto'
)

In [10]:
res

ChatCompletion(id='chatcmpl-9mOTgJIwVYyWmQ4tpChc14A5wKdCn', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_TpkD46WAPc5nh3RXmx8Rn6W4', function=Function(arguments='{"ticker":"ABEV3","periodo":"1d"}', name='retorna_cotacao_historica'), type='function')]))], created=1721320864, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=28, prompt_tokens=201, total_tokens=229))

In [15]:
tool_calls = res.choices[0].message.tool_calls

if tool_calls:
    mensagens.append(res.choices[0].message)

    for tool_call in tool_calls:
        func_name = tool_call.function.name
        function_to_call = funcoes_disponiveis[func_name]
        func_args = json.loads(tool_call.function.arguments)
        func_return = function_to_call(**func_args)

        mensagens.append({"tool_call_id": tool_call.id, "role": "tool", "name": func_name, "content": func_return})
        
    segunda_res = client.chat.completions.create(
		messages=mensagens,
		model='gpt-3.5-turbo'
	)
    
    mensagens.append(segunda_res.choices[0].message.content)

In [14]:
segunda_res.choices[0].message.content

'A cotação atual da Ambev (ABEV3) é de R$ 11.73.'